In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

# MultiIndex

In [3]:
idx_arr = [['a', 'a', 'b', 'b', 'c', 'c'], [1, 2, 1, 2, 1, 2]]
index = list(zip(*idx_arr))
index
multi_index = pd.MultiIndex.from_tuples(index, names=["first", "second"])
data = pd.Series(data=[1, 2, np.nan, 4, 5, 6], index=multi_index)
print(data.loc["b", 1:2])
print(data["b"][0:])

first  second
b      1         NaN
       2         4.0
dtype: float64
second
1    NaN
2    4.0
dtype: float64


In [4]:
data

first  second
a      1         1.0
       2         2.0
b      1         NaN
       2         4.0
c      1         5.0
       2         6.0
dtype: float64

In [5]:
ds = data.unstack() # to convert series to a dataframe, pivot table
print(type(ds))

<class 'pandas.core.frame.DataFrame'>


In [6]:
# multiindex on columns
data = pd.DataFrame(data=np.random.randint(0,10,size=16).reshape(4,4),
index = [["a", "a", "b", "b"], [1, 2, 1, 2, ]],
columns=[["ohio","ohio","or", "or"], ["green", "red", "green", "red"]])

In [7]:
data

ohio        or    
    green red green red
a 1     5   1     0   3
  2     4   3     0   3
b 1     8   3     6   1
  2     8   5     9   4

In [8]:
data.columns.names = ["state", "color"]
data.index.names = ["f", "s"]
data

state  ohio        or    
color green red green red
f s                      
a 1       5   1     0   3
  2       4   3     0   3
b 1       8   3     6   1
  2       8   5     9   4

In [9]:
data.swaplevel("f", "s")

state  ohio        or    
color green red green red
s f                      
1 a       5   1     0   3
2 a       4   3     0   3
1 b       8   3     6   1
2 b       8   5     9   4

In [10]:
data.swaplevel("state", "color", axis=1)

color green  red green red
state  ohio ohio    or  or
f s                       
a 1       5    1     0   3
  2       4    3     0   3
b 1       8    3     6   1
  2       8    5     9   4

In [11]:
data.sort_index(level=1)

state  ohio        or    
color green red green red
f s                      
a 1       5   1     0   3
b 1       8   3     6   1
a 2       4   3     0   3
b 2       8   5     9   4

In [12]:
data

state  ohio        or    
color green red green red
f s                      
a 1       5   1     0   3
  2       4   3     0   3
b 1       8   3     6   1
  2       8   5     9   4

In [13]:
data.reset_index() # makes named indexes as columns

state  f  s  ohio        or    
color       green red green red
0      a  1     5   1     0   3
1      a  2     4   3     0   3
2      b  1     8   3     6   1
3      b  2     8   5     9   4

In [14]:
data

state  ohio        or    
color green red green red
f s                      
a 1       5   1     0   3
  2       4   3     0   3
b 1       8   3     6   1
  2       8   5     9   4

In [15]:
data.groupby(level="f").sum()

state  ohio        or    
color green red green red
f                        
a         9   4     0   6
b        16   8    15   5

In [16]:
data.groupby("state", axis=1).sum()

state  ohio  or
f s            
a 1       6   3
  2       7   3
b 1      11   7
  2      13  13

In [17]:
np.random.seed(seed=42)
frame = pd.DataFrame({"a": range(7), "b": range(7, 0, -1),
                    "c": ["one", "one", "one", "two", "two",
                    "two", "two"],
                    "d": np.random.randint(1, 10, size=7,)})

frame

,a,b,c,d
0,0,7,one,7
1,1,6,one,4
2,2,5,one,8
3,3,4,two,5
4,4,3,two,7
5,5,2,two,3
6,6,1,two,7


In [18]:
frame1 = frame.set_index(["c", "d"]) # set columns as index

In [19]:
frame1

a  b
c   d      
one 7  0  7
    4  1  6
    8  2  5
two 5  3  4
    7  4  3
    3  5  2
    7  6  1

In [20]:
frame1 = frame1.sort_index(level="d")

In [21]:
frame1

a  b
c   d      
two 3  5  2
one 4  1  6
two 5  3  4
one 7  0  7
two 7  4  3
    7  6  1
one 8  2  5

In [22]:
frame1.index.levels

FrozenList([['one', 'two'], [3, 4, 5, 7, 8]])

In [23]:
frame1.index.nlevels

2

In [24]:
data = pd.Series(data=[1, 2, np.nan, 4, 5, 6], index=idx_arr)
data

data = pd.Series(data=[1, 2, np.nan, 4, 5, 6], index={'first':idx_arr[0], 'second': idx_arr[1]})
data

ValueError: Length of values (6) does not match length of index (2)

# Merge & Concat
- `merge` connects **rows** based on one or more keys
- `concat` stacks together objects along an axis
- `combine_first` splices overlapping data to fill in missing values
- `merge_asof` timeseries-based windows joins between DF objects

In [25]:
df1 = pd.DataFrame(
        {
           "A":  ["A0", "A1", "A2", "A3"],
           "B": ["B0", "B1", "B2", "B3"],
        "C": ["C0", "C1", "C2", "C3"],
            "D": ["D0", "D1", "D2", "D3"],
        },
        index=[0, 1, 2, 3],
    )
    

df2 = pd.DataFrame(
        {
            "A": ["A4", "A5", "A6", "A7"],
             "B": ["B4", "B5", "B6", "B7"],
             "C": ["C4", "C5", "C6", "C7"],
             "D": ["D4", "D5", "D6", "D7"],
        },
        index=[4, 5, 6, 7],
    )
    

df3 = pd.DataFrame(
        {
             "A": ["A8", "A9", "A10", "A11"],
             "B": ["B8", "B9", "B10", "B11"],
             "C": ["C8", "C9", "C10", "C11"],
             "D": ["D8", "D9", "D10", "D11"],
        },
        index=[8, 9, 10, 11],
    )
frames= [df1, df2, df3]

## Merge

In [26]:
df_merge = pd.merge(df1, df2, on=['A'], how="outer")
df_merge

,A,B_x,C_x,D_x,B_y,C_y,D_y
0,A0,B0,C0,D0,NaN,NaN,NaN
1,A1,B1,C1,D1,NaN,NaN,NaN
2,A2,B2,C2,D2,NaN,NaN,NaN
3,A3,B3,C3,D3,NaN,NaN,NaN
4,A4,NaN,NaN,NaN,B4,C4,D4
5,A5,NaN,NaN,NaN,B5,C5,D5
6,A6,NaN,NaN,NaN,B6,C6,D6
7,A7,NaN,NaN,NaN,B7,C7,D7


In [27]:
df1 = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "a", "b"],
"data1": pd.Series(range(7), dtype="Int64")})
df1 = df1.set_index("key")

df2 = pd.DataFrame({"key": ["a", "a", "b", "d"],
"data2": pd.Series(range(4), dtype="Int64")})
print(df1)
print(df2)

     data1
key       
b        0
b        1
a        2
c        3
a        4
a        5
b        6
  key  data2
0   a      0
1   a      1
2   b      2
3   d      3


In [28]:
pd.merge(df1, df2, on="key", how="right")

,key,data1,data2
0,a,2,0
1,a,4,0
2,a,5,0
3,a,2,1
4,a,4,1
5,a,5,1
6,b,0,2
7,b,1,2
8,b,6,2
9,d,<NA>,3


In [29]:
pd.merge(df1, df2, on="key", how="outer")

,key,data1,data2
0,b,0,2
1,b,1,2
2,b,6,2
3,a,2,0
4,a,2,1
5,a,4,0
6,a,4,1
7,a,5,0
8,a,5,1
9,c,3,<NA>


In [30]:
left = pd.DataFrame({"key1": ["foo", "foo", "bar"],
                         "key2": ["one", "two", "one"],
                         "val": pd.Series([1, 2, 3], dtype='Int64')})

right = pd.DataFrame({"key1": ["foo", "foo", "bar", "bar"],
                          "key2": ["one", "one", "one", "two"],
                          "val": pd.Series([4, 5, 6, 7], dtype='Int64')})
print(left)
print(right)
                    

  key1 key2  val
0  foo  one    1
1  foo  two    2
2  bar  one    3
  key1 key2  val
0  foo  one    4
1  foo  one    5
2  bar  one    6
3  bar  two    7


In [31]:
pd.merge(left, right, on=["key1", "key2"], sort=True, how="outer")                       

,key1,key2,val_x,val_y
0,bar,one,3,6
1,bar,two,<NA>,7
2,foo,one,1,4
3,foo,one,1,5
4,foo,two,2,<NA>


In [32]:
left = pd.DataFrame({"key1": ["foo", "foo", "bar"],
                         "key2": ["one", "two", "one"],
                         "val": pd.Series([1, 2, 3], dtype='Int64')})
left = left.set_index("key1")
                        

right = pd.DataFrame({"key1": ["foo", "foo", "bar", "bar"],
                          "key2": ["one", "one", "one", "two"],
                          "val": pd.Series([4, 5, 6, 7], dtype='Int64')})
print(left)
print(right)
                    

     key2  val
key1          
foo   one    1
foo   two    2
bar   one    3
  key1 key2  val
0  foo  one    4
1  foo  one    5
2  bar  one    6
3  bar  two    7


In [33]:
pd.merge(left, right, left_index=True, right_on="key1", sort=True, how="outer")

,key2_x,val_x,key1,key2_y,val_y
2,one,3,bar,one,6
3,one,3,bar,two,7
0,one,1,foo,one,4
1,one,1,foo,one,5
0,two,2,foo,one,4
1,two,2,foo,one,5


In [34]:
left1 = pd.DataFrame({"key": ["a", "b", "a", "a", "b", "c"],
                   "value": pd.Series(range(6), dtype="Int64")})

right1 = pd.DataFrame({"group_val": [3.5, 7]}, index=["a", "b"])
print(left1)
print(right1)

  key  value
0   a      0
1   b      1
2   a      2
3   a      3
4   b      4
5   c      5
   group_val
a        3.5
b        7.0


In [35]:
pd.merge(left1, right1, left_on="key", right_index=True, how="outer")

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


### Multi-index merging

In [36]:
lefth = pd.DataFrame({"key1": ["Ohio", "Ohio", "Ohio",
                                "Nevada", "Nevada"],
                       "key2": [2000, 2001, 2002, 2001, 2002],
                       "data": pd.Series(range(5), dtype="Int64")})

righth_index = pd.MultiIndex.from_arrays(
     [
         ["Nevada", "Nevada", "Ohio", "Ohio", "Ohio", "Ohio"],
         [2001, 2000, 2000, 2000, 2001, 2002]
     ]
 )

print(lefth)

     key1  key2  data
0    Ohio  2000     0
1    Ohio  2001     1
2    Ohio  2002     2
3  Nevada  2001     3
4  Nevada  2002     4


In [37]:
righth = pd.DataFrame({"event1": pd.Series([0, 2, 4, 6, 8, 10], dtype="Int64",
                                            index=righth_index),
                        "event2": pd.Series([1, 3, 5, 7, 9, 11], dtype="Int64",
                                            index=righth_index)})
print(righth)

             event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11


In [39]:
pd.merge(lefth, righth, left_on=["key1", "key2"], right_index=True, how="outer")

,key1,key2,data,event1,event2
0,Ohio,2000,0,4,5
0,Ohio,2000,0,6,7
1,Ohio,2001,1,8,9
2,Ohio,2002,2,10,11
3,Nevada,2001,3,0,1
4,Nevada,2002,4,<NA>,<NA>
4,Nevada,2000,<NA>,2,3


In [41]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                      index=["a", "c", "e"],
                      columns=["Ohio", "Nevada"]).astype("Int64")

right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                       index=["b", "c", "d", "e"],
                       columns=["Missouri", "Alabama"]).astype("Int64")
print(left2)
print(right2)

   Ohio  Nevada
a     1       2
c     3       4
e     5       6
   Missouri  Alabama
b         7        8
c         9       10
d        11       12
e        13       14


In [43]:
pd.merge(left2, right2, left_index=True, right_index=True, how="outer")

,Ohio,Nevada,Missouri,Alabama
a,1,2,<NA>,<NA>
b,<NA>,<NA>,7,8
c,3,4,9,10
d,<NA>,<NA>,11,12
e,5,6,13,14


### `join` to simplify merging by index

In [45]:
left2.join(right2, how="outer")

,Ohio,Nevada,Missouri,Alabama
a,1,2,<NA>,<NA>
b,<NA>,<NA>,7,8
c,3,4,9,10
d,<NA>,<NA>,11,12
e,5,6,13,14


In [46]:
print(left1, "\n", right1)

  key  value
0   a      0
1   b      1
2   a      2
3   a      3
4   b      4
5   c      5 
    group_val
a        3.5
b        7.0


In [48]:
left1.join(right1, on="key", sort=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


In [55]:
right2

,Missouri,Alabama
b,7,8
c,9,10
d,11,12
e,13,14


In [58]:
another = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
   index=["a", "c", "e", "f"],
   columns=["New York", "Oregon"])

left3 = left1.set_index("key")
left3.join([right2, another], how="outer")
# left2.join([right2, another], how="outer")


,value,Missouri,Alabama,New York,Oregon
a,0,<NA>,<NA>,7.0,8.0
a,2,<NA>,<NA>,7.0,8.0
a,3,<NA>,<NA>,7.0,8.0
b,1,7,8,NaN,NaN
b,4,7,8,NaN,NaN
c,5,9,10,9.0,10.0
d,<NA>,11,12,NaN,NaN
e,<NA>,13,14,11.0,12.0
f,<NA>,<NA>,<NA>,16.0,17.0


## Concat

### on numpy arrays

In [124]:
pieces = {"x": df1, "y": df2, "z": df3}
r = pd.concat(pieces, levels=[["z", "x", "y", "w"]], keys=["x", "z"], names=["group_key"])
r

A    B    C    D
group_key                       
x         0    A0   B0   C0   D0
          1    A1   B1   C1   D1
          2    A2   B2   C2   D2
          3    A3   B3   C3   D3
z         8    A8   B8   C8   D8
          9    A9   B9   C9   D9
          10  A10  B10  C10  D10
          11  A11  B11  C11  D11

# Pivoting
- `stack` pivots from columns into rows
- `unstack` pivots from rows into columns

In [125]:
??df1.reindex


Signature:
df1.reindex(
    labels=None,
    index=None,
    columns=None,
    axis=None,
    method=None,
    copy=True,
    level=None,
    fill_value=nan,
    limit=None,
    tolerance=None,
)
Docstring:
Conform Series/DataFrame to new index with optional filling logic.

Places NA/NaN in locations having no value in the previous index. A new object
is produced unless the new index is equivalent to the current one and
``copy=False``.

Parameters
----------

keywords for axes : array-like, optional
    New labels / index to conform to, should be specified using
    keywords. Preferably an Index object to avoid duplicating data.

method : {None, 'backfill'/'bfill', 'pad'/'ffill', 'nearest'}
    Method to use for filling holes in reindexed DataFrame.
    Please note: this is only applicable to DataFrames/Series with a
    monotonically increasing/decreasing index.

    * None (default): don't fill gaps
    * pad / ffill: Propagate last valid observation forward to next
      valid.
    

In [127]:
2 // 3.0

0.0